# Least squares approximation


_MATH 420_ <br>
_Section 5.1_ <br>
__Spring 2021__ <br>


Let's write Julia code that finds the least squares fit to a polynomial of at most degree $deg$ to points in $\bf{R}^2$, where the user can choose the degree.

Our textbook builds the normal equations using loops (pages 174 and 175), but we'll use a matrix approach that eliminates the loops. 

Do we want to be careful? Of course we do. Thus, along with the coefficients of the polynomial, we'll return the infinity norm condition number of the coefficient matrix of the normal equations. The condition number will warn us when the solution to the normal equations is possibly inaccurate.

Here is our code--it returns an $deg+1$ member array $[c_0, c_1, \dots, c_{deg}]$, where the best fit polynomial is
$$
   x \mapsto c_0 + c_1 x + c_2 x^2 + \cdots + c_n x^n.
$$

We'll tell Julia that the first two inputs are one dimensional arrays of _real_ numbers; for data with complex numbers, we would need to modify our method.

In [ ]:
using LinearAlgebra,Gadfly

In [ ]:
function least_squares_fit(x::Array{Real,1}, y::Array{Real,1}, deg::Integer) 
    m = [x[i]^j for i=1:length(x), j=0:deg] #Julia is OK with 0^0
    y = transpose(m) * [y[i] for i=1:length(y),j=1:1]
    m = transpose(m)*m
    m \ y, cond(m,Inf) #return both solution & infinity norm condition number
end;

Let's test our code using data that _exactly_ fits the polynomial
$$
 P = x  \mapsto -64 + 56 x - 14 x^2 + x^3 .
$$
For the knots, we'll use the members of the following array:

In [ ]:
x = [1, 1.2, 1.9, 2, 2.7, 2.9, 3.1, 3.9, 4, 4.2, 4.4, 4.5];

The corresponding second coordinates are the values of our polynomial $P$ applied to the knots

In [ ]:
y = map(x -> -64 + 56*x - 14*x^2 + x^3, x);

The coefficients of the least squares fit polynomial are

In [ ]:
@time c = least_squares_fit(x,y, 3)

 The coefficients of the fitted polynomial are close to the expected values of -64, 56, -14, 1. The matrix condition number is about $3 \times 10^6$, so we expect some of the digits to differ from their true values, and they do.

We'll extract the first member of `c`.  To define a function that evaluates the lest squares fit (LSF)polynomial, we'll use `@evalpoly`

In [ ]:
c = first(c);

In [ ]:
P = x -> @evalpoly(x, c...)

Because we constructed the array `y` by mapping $x  \mapsto -64 + 56 x - 14 x^2 + x^3$ over the array `x`, we expect the differences $P(x_k) - y_k$ to be small. They are. Let's declare success.

In [ ]:
map(P,x) - y

Great--the fitted polynomial agrees with the data. How about a graphical verification?

In [ ]:
plot(layer(x=x,y=y,Geom.point,color=[colorant"black"]),
     layer(P, minimum(x), maximum(x), color=[colorant"purple"]))

Let's use some random numbers to make the data a bit wiggly. The function `rand` returns a random number in the interval $[0,1)$, so adding `4*(rand(Float64) - 1/2)` moves each second coordinate up or down at most 2.

In [ ]:
y = map(x -> -64 + 56*x - 14*x^2 + x^3 + 4*(rand(Float64) - 1/2), x);

Again, let's fit this data to  a third degree polynomial.

In [ ]:
c = least_squares_fit(x,y, 3)

Again, the matrix condition number is about $3 \times 10^6$. Compared to the machine epsilon, we expect about 10 correct digits.

In [ ]:
c = first(c)

In [ ]:
P = x -> @evalpoly(x, c...)

In [ ]:
plot(layer(x=x,y=y,Geom.point, color=[colorant"black"]),
     layer(P, minimum(x), maximum(x), color=[colorant"purple"]))

And we can fit to a degree eleven polynomial too:

In [ ]:
c = least_squares_fit(x,y, 11)

But the condition number is about $8 \times 10^{24}$! Generally for a huge condition number, we expect the least squares fit polynomial to fit the data quite well. But we also expect that are are other polynomials, some with wildly differing coefficients, with the same or smaller degree that fit the data _just as well_. 


<div class="alert alert-block alert-warning"><b>Warning:</b>  Checking the matrix condition number is required.</div>

In [ ]:
c = first(c)

In [ ]:
P = x -> @evalpoly(x, c...)

In [ ]:
plot(layer(x=x,y=y,Geom.point,color=[colorant"black"]),
     layer(P, minimum(x), maximum(x), Geom.line,  color=[colorant"purple"]))

Now let's convert to `BigFloat` numbers and again fit also to a degree 11 polynomial

In [ ]:
xx = map(s -> convert(BigFloat,s),x);

In [ ]:
yy = map(s -> convert(BigFloat,s),y);

In [ ]:
cc = least_squares_fit(xx,yy, 11)

In [ ]:
cc = first(cc)

In [ ]:
PP = x -> @evalpoly(x, cc...)

Are the polynomial coefficients nearly the same as when we used `Float64` numbers? Not even close!

In [ ]:
c - cc

Are the polynomials $P$ and $PP$ close? Well, yes at the knots, in between, not so much.

In [ ]:
plot(layer(x-> P(x) - PP(x),1,4,color=[colorant"purple"]))

In [ ]:
plot(layer(x=x,y=y,Geom.point,color=[colorant"black"]),
     layer(P, minimum(x), maximum(x), Geom.line, color=[colorant"purple"]),
     layer(PP, minimum(x), maximum(x),  Geom.line, color=[colorant"green"]))

What's the story? We've found two degree 11 polynomials that both fit the data very well, _but_ the two polynomials differ considerably. So it goes--there are _many_ 11 degree polynomials that fit the data well. 

The huge condition number tells us that we shouldn't expect that the data determines the polynomial coefficients all that well. As we have seen before, the huge condition number _doesn't_ in this case give us a solution that is a poor fit to the data. Using either `Float64` or `BigFloats` the fit is OK, but visually, the green curve (determined using `BigFloats` is the better fit).

If this was an experiment and our Nobel Prize hinged on an accurate determination of the polynomial coefficients,  we've done the wrong experiment.



## Fit to nonpolynomial basis

There is nothing magic about fitting data to a polynomial. We can fit data to using a linear combination (LC) of any basis function. For the set of _functions_
$$
   \{f_1, f_2, \dots f_n \}
$$
Let's find numbers $c_1, c_2, \dots, c_n$ such that the LC $F = c_1 f_1 +c_2 f_2 + \cdots c_n f_n$ minimizes the function 
$$
     (c_1,c_2, \dots, c_n) \mapsto \sum(|F(x_k) - y_k|^2,
$$
where the numbers $x_1, x_2, \dots x_n$ and  $y_1, y_2, \dots y_n$ are given. Following the same process as for the polynomial case, we define the matrix
$$
    M_{i,j} = f_j(x_k)
$$
and solve $M^T M \mathbf{c} =  M^T \mathbf{y}$. To modify our code, we only need to pass an array of functions and change the way the matrix `M` is constructed.

In [ ]:
function least_squares_fit2(x::Array{Real,1}, y::Array{Real,1}, fn::Array) 
    d = length(fn)
    m = [fn[j](x[i]) for i=1:length(x), j=1:d]
    y = transpose(m) * [y[i] for i=1:length(y),j=1:1]
    m = (transpose(m) * m)
    m \ y, cond(m,Inf)
end;

Here is a fun example.  We might expect that the average monthly temperature in Kearney, Nebraska to approximately have the form
$$
   T(x) = c_0 + c_1 \cos(2 \pi x / 12) + c_2 \sin(2 \pi x / 12),
$$
where $x$ is the month with January being month $0$ and December month $11$. That makes month $12$ also correspond to January.  That is, we assume the temperature is sinusoidal.

In [ ]:
month = [0,1,2,3,4,5,6,7,8,9,10,11];

For the average monthly temperature, we'll use the data from https://www.timeanddate.com/weather/usa/kearney/climate.
For the average high by month, we have

In [ ]:
Temp = [37, 41,53,63,73,83,87,85,78,66,51,39];

The basis functions are

In [ ]:
fn = [x -> 1.0, x -> cos(2*pi * x/12), x -> sin(2*pi*x/12)];

The condition number is small, so we have a good solution.

In [ ]:
c = least_squares_fit2(month, Temp, fn)

In [ ]:
c = first(c)

In [ ]:
fittedTemp = x -> c[1] * fn[1](x)  +  c[2] * fn[2](x) +  c[3] * fn[3](x)

Visually, the fit is good. 

__Funny:__ My first effort, I miss typed the December high as 34 instead of 39. I was disappointed with the fit of the function for December. Ha! It was faulty data, not a faulty method. There is a theoretical basis for assuming that the temperature data is approximately sinusoidal.

In [ ]:
plot(layer(x=month,y=Temp,Geom.point,color=[colorant"black"]),
     layer(fittedTemp, -2, 14, Geom.line, color=[colorant"purple"]))
   

According to the fitted function, what days of the year have a local extrema? Thus we need to solve the equation
$$
  \mbox{fittedTemp}^\prime(x) = 0
$$
We could solve this equation numerically, but let's apply some calculus I instead. We need to solve
$$
   c_2 \cos(\pi x/6) + c_3 \sin(\pi x /6) = 0
$$
Thus
$$
    x = \frac{6}{\pi} \left(\tan^{-1} (c_3/c_2) + k \pi \right),
$$
where $k$ is an integer.  Specifically


In [ ]:
(6/pi) * (atan(c[3]/c[2])), (6/pi) * (atan(c[3]/c[2]) + pi)

The day $0.07$ is about 2 January; and the day $6.07$ is about 2 July.

##  Theory

Do the normal equations _always_ have a solution?  Yes, but the solution needn't be unique. If the columns of $A$ are linearly dependent, the coefficient matrix of the normal equations does not have an inverse and there are infinitly many solutions to the normal equations. An example is
$$
  \begin{bmatrix} 1 & 	2 \\  2 &	4 \end{bmatrix} \begin{bmatrix} x \\ y \end{bmatrix} = \begin{bmatrix} 1 \\ 8 \end{bmatrix}.
$$
The normal equations are
$$
  \begin{bmatrix}5 & 10\\10 & 20\end{bmatrix} \begin{bmatrix} x \\ y \end{bmatrix}  = \begin{bmatrix}17\\ 34\end{bmatrix}.
$$
The solution to the normal equations is
$$
  x=\frac{17}{5}-2 r ,y= r,
$$
where $r$ is arbitrary.  It is possible to choose $r$ to minimize $\| \langle x, y \rangle \| $. With that choice, the solution is unique.

## Efficiency

For each member of the $d \times d$ matrix $M^T M$, we need to do a dot product of two vectors with $n$ components, where $n$ is the number of knots. Thus the number of additions and multiplications needed to construct the coefficient matrix is $2 d^2 n^2 $.  The effort required to solve the resulting $d \times d$ matrix is about $d^3$, but generally $d$ is modestly sized but $n$ might be huge, say $n = 10^6$. Thus the spendy part isn't solving the equations, but rather constructing the coefficient matrix for the normal equations.  Together the cost of fitting $n$ data points to a polynomial of degree $d$ is about $d^3 + 2 d^2 n^2$

There is a clever method for solving least squares problems that works with the matrix $M$ without constructing $M^T M$.  The method involves factoring $M$ into a product of three matrices called the _singular value decomposition_ (SVD). But the cost of computing the SVD for a $n \times d$  matrix
is $\mathrm{O}(n^2 d+n d^2+d^3) $, so the effort to compute the SVD decomposition is about the same as computing the normal equations. For a modest sized $d$ and large $n$, the cost for both methods is proportional to $n^2$.

One feature of the SVD method is that for the case of a non unique solution (linearly dependent matrix columns), the SVD method chooses a specific solution. Another method that doesn't require forming the normal equations is the QR factorization.

Sometimes the normal equation method is criticized because $\mbox{cond}_{M^T M} = (\mbox{cond}_{M})^2$. So if $\mbox{cond}_{M}$ is big, $\mbox{cond}_{M^T M}$ might be huge. So it might seem that a method that circumvents forming $M^T M$ might be an advantage. All that's true, but that argument ignores the additonal complications from computing the SVD or QR factorizations. Actually, in the next section we'll see there is a simple method that often can greatly reduce the condition number of $\mbox{cond}_{M^T M}$.

The theory of both the SVD and QR factorizations are involved, and the topic is better suited for an advanced class. Although we could use the SVD as a black box and use it without understanding its theory, let's stick with the maxim:

 “A little learning is a dangerous thing.”  Alexander Pope, _An Essay on Criticism,_ 1709.

## Reducing the condition number


Sometimes a change of basis can reduce a condition number greatly. This allows for determining the fitted function more accurately.  We'll use an example from the well-known textbook  _Elementary Numerical Analysis: An Algorithmic Approach_, by Conte and Carl de Boor. (First edition was 1965, I think.)

What do we mean by a change of basis? If we are fitting to a second degree polynomial, we can take our polynomial to be a linear combination of the functions $\{x \mapsto 1, x \mapsto x, x \mapsto x^2 \}$. But we could also use a linear combination of $\{x \mapsto 1, x \mapsto x-1/2, x \mapsto (x-1/3)(x-2/3) \}$, for example. Both of these sets of functions form a basis for the vector space of polynomials of degree two or less.

The example, although 55 years old, still works:

In [ ]:
x = [10 + (k-1)/5 for k = 1:6]

In [ ]:
y = map(s -> 1+ 10.0*s+ 0.1*s^2 + 0.001*(rand(Float64)-1/2),x)

Using the basis $\{x \mapsto 1, x \mapsto x, x \mapsto x^2 \}$, the least squares fit is

In [ ]:
c = least_squares_fit(x,y,2)

But the condition number is about $2 \times 10^{10}$.  Although the fitted polynomial fits the data OK, we expect the coefficients to differ from their true values. And they do differ from $1,10, 0.1$

In [ ]:
c = first(c)

In [ ]:
P = x -> @evalpoly(x, c...)

In [ ]:
plot(layer(x=x,y=y,Geom.point, color=[colorant"black"]),
     layer(P, minimum(x), maximum(x), color=[colorant"purple"]))

In [ ]:
fn = [s -> 1.0, s -> s-10.5, s -> (s-10.3)*(s-10.7)];

Wow--we reduced the condition number from about $2 \times 10^{10}$ to a tiny $116$.  We are drinking from the fountain of success.

In [ ]:
c = least_squares_fit2(x,y,fn)

In [ ]:
c = first(c)

In [ ]:
P = s -> c[1]*fn[1](s)+ c[2]*fn[2](s) + c[3]*fn[3](s)

In [ ]:
plot(layer(x=x,y=y,Geom.point, color=[colorant"black"]),
     layer(P, minimum(x), maximum(x), color=[colorant"purple"]))

Explicitly, our polynomial is
$$
0.1020208832697176*(x-10.7)*(x-10.3)+12.10021076827413*(x-10.5)+117.0288365061491
$$
Expanding and collecting terms, it is
$$
0.1020208832697176*x^2+9.95777221961006*x+1.220344984426275
$$


Another example:  For least squares fit to a line, the condition number of the coefficient matrix involves
$$
   \frac{1}{ n \sum x_k^2  - \left(\sum x_k \right)^2}
$$
That factor is big with all the knots are about the same.  Let's try

In [ ]:
x = [1.0e6 + k / 100 for k = 0:100];

In [ ]:
y = map(q -> 0.1 + 0.390625*q,x);

In [ ]:
least_squares_fit(x,y,1)

The basis $\{ q -> 1.0, q ->  q - 1000000.5 \}$ reduces the condition number from $10^{25}$ to about $12$. That allows for an accurate solution

In [ ]:
fn = [ q -> 1.0, q ->  q - 1000000.5 ];

In [ ]:
least_squares_fit2(x,y,fn)

The best fit line is
$$
   x \mapsto 390625.29531250073 + 0.39062499998304995(x - 1000000.5).
$$
Expanding this we get
$$
  x \mapsto 0.3906249999830499 x + 0.1000169508042745.
$$
This compares well with the true solution $x  \mapsto 0.1 + 0.390625 x $.

<div class="alert alert-block alert-warning"><b>Fact:</b>  Fancy methods such as the QR or SVD factorizations aren't always needed for an accurate solution.</div>